<a href="https://colab.research.google.com/github/pach28/Forecasting-Sticker-Sales/blob/main/sticker_forecast_kaggle_stationary_treatment_%26_weighted_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

sns.set_style('whitegrid')

In [10]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn._oldcore")

In [11]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [12]:
import os

# Download the competition dataset
playground_series_s5e1_path = kagglehub.competition_download('playground-series-s5e1')

# Extract the directory name from the downloaded path
dataset_dir = os.path.splitext(playground_series_s5e1_path)[0]

# Construct the correct paths to the train.csv and test.csv files
train_csv_path = os.path.join(dataset_dir, 'train.csv')
test_csv_path = os.path.join(dataset_dir, 'test.csv')

# Read the CSV files using the correct paths
train_df = pd.read_csv(train_csv_path, parse_dates=["date"])
test_df = pd.read_csv(test_csv_path, parse_dates=["date"])

In [13]:
def get_gdp_per_capita(countries, start_year, end_year):
    # Construir la URL base de la API
    base_url = "https://api.worldbank.org/v2/country/"

    # Crear una lista de URLs para cada país
    urls = [f"{base_url}{country}/indicator/NY.GDP.PCAP.CD?date={start_year}:{end_year}&format=json" for country in countries]

    # Hacer las solicitudes a la API y crear un DataFrame para cada país
    dfs = []
    for url in urls:
        response = requests.get(url)
        data = response.json()

        # Check if the response is valid and contains data
        if response.status_code == 200 and len(data) > 1 and isinstance(data[1], list) and len(data[1]) > 0:
            df = pd.DataFrame(data[1])

            # Ensure 'country' and 'date' contain scalar values before pivoting
            # Example: If 'country' is a dictionary, extract a specific key
            if isinstance(df['country'].iloc[0], dict):
                df['country'] = df['country'].apply(lambda x: x['value'] if isinstance(x, dict) and 'value' in x else x)
            # Similarly, handle 'date' if necessary

            df = df.pivot_table(index='country', columns='date', values='value')
            dfs.append(df)
        else:
            print(f"Warning: Request for {url} failed or returned unexpected data.")

    # Concatenar los DataFrames
    df_final = pd.concat(dfs, axis=0)

    return df_final
countries = ['CAN', 'FIN', 'ITA','KEN','NOR','SGP']
df_pib = get_gdp_per_capita(countries, 2010, 2020)

In [14]:
#sacar el ponderado de gdp por pais siendo por ano, el gdp del pais/ la suma de todos los paises
df_pib_ponderado = df_pib.div(df_pib.sum())
df_pib_ponderado

date,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
country,,,,,,,,,,,
Canada,0.178301,0.175045,0.178867,0.175479,0.173632,0.175439,0.171860,0.171772,0.164793,0.169793,0.169942
Finland,0.174349,0.171145,0.161484,0.165654,0.170609,0.171278,0.176478,0.175408,0.175823,0.177138,0.190594
Italy,0.135654,0.130223,0.119660,0.119171,0.121808,0.122289,0.126420,0.123907,0.123035,0.123858,0.125263
Kenya,0.004092,0.003674,0.004364,0.004570,0.005052,0.005993,0.006312,0.006347,0.006503,0.007181,0.007524
Norway,0.330517,0.339278,0.346987,0.345215,0.332766,0.301063,0.287829,0.289772,0.293166,0.279969,0.266753
Singapore,0.177087,0.180635,0.188638,0.189911,0.196132,0.223938,0.231100,0.232794,0.236680,0.242061,0.239924
